In [1]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from itertools import *
from gurobipy import *

import gurobipy as gp
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
%matplotlib inline
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()
import math
import datetime
import os

In [2]:
#Funciones traduccion de resultados MILP
#Funcion de seleccion de maquina
def seleccionMaquina(maqDis):
    mD = maqDis.copy()
    if len(mD) > 0:
        fV = max(mD['fVelocidad'])
        m = mD[mD['fVelocidad'] == fV].iloc[0]['Maquina']
        mD.drop(mD[mD['Maquina'] == m].index, inplace = True)
    else:
        fV = 'x'
        m = 'No hay maquina'
    return [m, fV, mD]         

#Algoritmo heuristico
def dispRule(rutaIns, rutaSol, instancia):
    startTime = time.time()
    #Lectura de parametros
    tProcesamiento = pd.read_csv(rutaIns + '/tProcesamiento.csv')
    fVelocidad = pd.read_csv(rutaIns + '/fVelocidad.csv')
    tDistancia = pd.read_csv(rutaIns + '/tDistancias.csv')
    inalcanzable = pd.read_csv(rutaIns + '/inalcanzable.csv')
    parametros = pd.read_csv(rutaIns + '/parametros.csv', index_col = 'parametro')
    
    #Creacion archivo de resultados
    resultados = pd.DataFrame(columns = ['Dia', 'Maquina', 'Bloque1', 'Aplicacion1', 'Bloque2', 'Aplicacion2'])

    #Inicializacion trabajos pendientes
    Jpend = tProcesamiento[1:]
    dia = 0

    while len(Jpend) > 0:
        #Seleccion del dia
        if min(Jpend['dFin']) > dia:
            dia = min(Jpend['dFin'])
        else:
            dia = dia + 1

        #Seleccion de la maquina
        maq = seleccionMaquina(fVelocidad)[0]
        fVel = seleccionMaquina(fVelocidad)[1]
        maqDisp = seleccionMaquina(fVelocidad)[2]
        if maq == 'No hay maquina':
            acabo = True
        else:
            horasDisp = 8
            acabo = False


        #Creacion de trabajos    
        Jpos = Jpend[Jpend['dInicio'] <= dia]
        b1 = tProcesamiento.iloc[0]['Bloque']
        h1 = tProcesamiento.iloc[0]['Aplicacion']
        while (len(Jpos) > 0) & (not acabo):
            dist = tDistancia[(tDistancia['Bloque1'] == b1)]
            Jpos = Jpos.merge(dist, how = 'left', left_on = 'Bloque', right_on = 'Bloque2')
            jprime = Jpos[Jpos['Distancia'] == min(Jpos['Distancia'])].iloc[0]
            imp = len(inalcanzable[(inalcanzable['Maquina'] == maq) & (inalcanzable['Bloque'] == jprime['Bloque'])])

            if (horasDisp - (jprime['tProcesamiento']/fVel) - jprime['Distancia'] >= 0) & (imp == 0):
                horasDisp = horasDisp - (jprime['tProcesamiento']/fVel) - jprime['Distancia']
                b2 = jprime['Bloque']
                h2 = jprime['Aplicacion']
                resultados = resultados.append(pd.DataFrame({'Dia':[dia], 'Maquina':[maq], 'Bloque1':[b1], 
                                                             'Aplicacion1':[h1], 'Bloque2':[b2], 'Aplicacion2':[h2]}))
                Jpos.drop(Jpos[(Jpos['Bloque'] == b2) & (Jpos['Aplicacion'] == h2)].index, inplace = True)
                Jpend.drop(Jpend[(Jpend['Bloque'] == b2) & (Jpend['Aplicacion'] == h2)].index, inplace = True)
                b1 = b2
                h1 = h2

            else:
                b2 = tProcesamiento.iloc[0]['Bloque']
                h2 = tProcesamiento.iloc[0]['Aplicacion']
                resultados = resultados.append(pd.DataFrame({'Dia':[dia], 'Maquina':[maq], 'Bloque1':[b1], 
                                                             'Aplicacion1':[h1], 'Bloque2':[b2], 'Aplicacion2':[h2]}))
                maq = seleccionMaquina(maqDisp)[0]
                fVel = seleccionMaquina(maqDisp)[1]
                maqDisp = seleccionMaquina(maqDisp)[2]
                if maq == 'No hay maquina':
                    acabo = True
                else:
                    horasDisp = 8
                b1 = b2
                h1 = h2
            Jpos.drop(list(tDistancia.columns), axis = 1, inplace = True)
        if maq != 'No hay maquina':
            b2 = tProcesamiento.iloc[0]['Bloque']
            h2 = tProcesamiento.iloc[0]['Aplicacion']
            resultados = resultados.append(pd.DataFrame({'Dia':[dia], 'Maquina':[maq], 'Bloque1':[b1], 
                                                         'Aplicacion1':[h1], 'Bloque2':[b2], 'Aplicacion2':[h2]}), ignore_index = True)

    resultados = resultados.merge(tDistancia, how = 'left', on = ['Bloque1','Bloque2'])
    resultados['Distancia'] = resultados['Distancia'].apply(lambda x: x/parametros['valor']['Q'])
    resultados = resultados.merge(tProcesamiento, how = 'left', left_on = ['Bloque1','Aplicacion1'],
                                  right_on = ['Bloque','Aplicacion'])
    resultados.drop(['Bloque','Aplicacion'], axis = 1, inplace = True)
    resultados = resultados.merge(fVelocidad, how = 'left', on = ['Maquina'])
    resultados['ProcEfectivo'] = resultados['tProcesamiento']/resultados['fVelocidad']
    resultados['Earl-Tard'] = np.maximum(resultados['Dia']-resultados['dFin'],0) + np.maximum(resultados['dInicio']-resultados['Dia'],0)
    resultados['Earl-Tard'] = np.where(resultados['Bloque1'] == 'Bloque 0', 0, resultados['Earl-Tard'])
        
    tiempo = round(time.time() - startTime)
    FO = parametros['valor']['lambda1']*sum(resultados['Earl-Tard']) + parametros['valor']['lambda2']*sum(resultados['Distancia'])
    resultados.to_csv(rutaSol + '/resultados Drule.csv', index = False)
    return pd.DataFrame({'Instancia':[instancia], 'FO':[FO], 'Tiempo':[tiempo]})

#Correr modelo:
def run(tam, nInstancias):
    metricas = pd.DataFrame()
    rutaIns = 'Instancias/' + tam
    rutaSol = 'Resultados/' + tam
    
    if not os.path.isdir(rutaSol):
        os.mkdir(rutaSol)
        
    for inst in range(1, nInstancias+1):
        instancia = '/Instancia ' + str(inst)
        rutaIns = 'Instancias/' + tam + instancia
        rutaSol = 'Resultados/' + tam + instancia

        if not os.path.isdir(rutaSol):
            os.mkdir(rutaSol)
            
        
        res = dispRule(rutaIns, rutaSol, instancia[1:])
        metricas = metricas.append(res, ignore_index = True)
            
    metricas.to_csv('Resultados/' + tam + '/metricas Drule.csv', index = False)
    return metricas

In [3]:
run('Pequena', 10)

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:88: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:88: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:88: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:88: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:125: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprec

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:88: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:88: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is depreca

,Instancia,FO,Tiempo
0,Instancia 1,1.016315,1
1,Instancia 2,0.484223,1
2,Instancia 3,0.914295,1
3,Instancia 4,0.679626,1
4,Instancia 5,1.316859,1
5,Instancia 6,0.689994,1
6,Instancia 7,0.744701,2
7,Instancia 8,0.839946,1
8,Instancia 9,0.963714,1
9,Instancia 10,1.157857,1


In [4]:
run('Mediana', 10)

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:73: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:73: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:88: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:88: FutureWarning:

The frame.append method is depreca

...

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:73: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:88: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:88: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:88: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_40235/152710146.py:63: FutureWarning:

The frame.append method is depreca

,Instancia,FO,Tiempo
0,Instancia 1,2.805415,4
1,Instancia 2,1.668910,4
2,Instancia 3,1.147457,3
3,Instancia 4,1.514777,3
4,Instancia 5,1.616100,3
5,Instancia 6,1.151994,2
6,Instancia 7,1.778396,3
7,Instancia 8,1.839788,3
8,Instancia 9,2.167967,2
9,Instancia 10,2.037363,3


In [3]:
run('Grande', 5)

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:73: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:73: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:88: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is depreca

...

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:88: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:73: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:73: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:88: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_48885/152710146.py:63: FutureWarning:

The frame.append method is depreca

,Instancia,FO,Tiempo
0,Instancia 1,5.611008,6
1,Instancia 2,7.046090,5
2,Instancia 3,6.188083,5
3,Instancia 4,3.114076,4
4,Instancia 5,3.818006,3


In [3]:
run('Real', 1)

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:73: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is depreca

...

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:73: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is depreca

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/wq/9r9y5cp52f9_q3yrjtwy1_d80000gn/T/ipykernel_51039/152710146.py:63: FutureWarning:

The frame.append method is depreca

,Instancia,FO,Tiempo
0,Instancia 1,33640.390994,95
